## GPT-2를 이용한 KorNLI분류
KorNLI 데이터는 카카오 브레인에서 공개한 한국어 벤치마크 데이터셋입니다.
KorNLI 데이터셋의 깃허브: https://github.com/kakaobrain/KorNLUDatasets
NLI(Natural Language Inferencing) 는 두 개의 문장이 주어지고, 두 개의 문장이 수반 (entailment) 관계
인지, 모순 (contradiction) 관계인지, 중립 (neutral) 관계인지를 맞추는 문제입니다. 이번에는 한국어
NLI 데이터셋인 KorNLI 데이터셋을 가지고 두 개의 문장을 입력받아서 세 개 중 하나인 관계를 맞추는 다
중 클래스 분류 문제를 풀어봅시다. 그리고 BERT 가 두 개의 문장을 입력받을 때는 어떤 식으로 전처리를
진행하는지 보겠습니다.

In [1]:
!pip install transformers

###1. 데이터 로드 및 정제

In [2]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
from sklearn import preprocessing
from transformers import AutoTokenizer, TFGPT2Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

훈련 데이터, 검증 데이터, 테스트 데이터로 사용할 NLI 데이터를 로드합니다.

In [3]:
# 훈 련 데 이 터 다 운 로 드
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/multinli.train.ko.tsv", filename="multinli.train.ko.tsv")
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/snli_1.0_train.ko.tsv", filename="snli_1.0_train.ko.tsv")
# 검 증 데 이 터 다 운 로 드
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/xnli.dev.ko.tsv", filename="xnli.dev.ko.tsv")
# 테 스 트 데 이 터 다 운 로 드
urllib.request.urlretrieve("https://raw.githubusercontent.com/kakaobrain/KorNLUDatasets/master/KorNLI/xnli.test.ko.tsv", filename="xnli.test.ko.tsv")

('xnli.test.ko.tsv', <http.client.HTTPMessage at 0x7fbcb6383f90>)

훈련 데이터로 사용할 두 개의 파일과, 검증 데이터와 테스트 데이터 파일을 각각 데이터프레임으로 로드
합니다.

In [4]:
# sep=\t tab기준 구분, quoting=3-> 인용부호 무시
train_snli = pd.read_csv("snli_1.0_train.ko.tsv", sep='\t', quoting=3)
train_xnli = pd.read_csv("multinli.train.ko.tsv", sep='\t', quoting=3)
val_data = pd.read_csv("xnli.dev.ko.tsv", sep='\t', quoting=3)
test_data = pd.read_csv("xnli.test.ko.tsv", sep='\t', quoting=3)

훈련 데이터로 사용할 두 개의 데이터프레임을 학습을 위해 하나로 합쳐줍니다. 그리고 데이터를 섞어줍
니다.

In [5]:
# 결 합 후 섞 기
train_data = pd.concat([train_snli, train_xnli])
train_data = train_data.sample(frac=1)

In [6]:
train_data.head()

,sentence1,sentence2,gold_label
254703,두 아이가 텐트 뒤에 있는 흙밭을 뛰어다닌다.,두 아이가 밖에서 술래잡기를 하고 있다.,neutral
176548,비대칭적인 구성과 신비로운 황홀경의 분위기가 그의 작품을 상징한다.,그의 작품은 비대칭적인 구성과 신비로운 황홀경의 분위기로 특징지어진다.,entailment
487320,"파우더블루 셔츠와 카키스, 회색 테니스화를 신은 남자가 남색 스카프, 회색 치마, ...",한 말벌 남자가 이슬람 여성과 그녀의 아들 옆에 앉아 있다.,neutral
185084,갈색 스웨터를 입은 금발 소년이 화려한 우산에 통증을 느낀다.,갈색 스웨터를 입은 금발 소년이 우산을 부수고 있다.,contradiction
21339,밝은 연들이 산을 배경으로 날고 있다.,사람들이 연날리기 대회에 참가하고 있다,neutral


In [7]:
val_data.head()

,sentence1,sentence2,gold_label
0,"그리고 그가 말했다, ""엄마, 저 왔어요.""",그는 학교 버스가 그를 내려주자마자 엄마에게 전화를 걸었다.,neutral
1,"그리고 그가 말했다, ""엄마, 저 왔어요.""",그는 한마디도 하지 않았다.,contradiction
2,"그리고 그가 말했다, ""엄마, 저 왔어요.""",그는 엄마에게 집에 갔다고 말했다.,entailment
3,내가 무엇을 위해 가고 있는지 또는 어떤 것을 위해 있는지 몰랐기 때문에 워싱턴의 ...,나는 워싱턴에 가본 적이 없어서 거기 배정을 받았을 때 그 장소를 찾으려다가 길을 ...,neutral
4,내가 무엇을 위해 가고 있는지 또는 어떤 것을 위해 있는지 몰랐기 때문에 워싱턴의 ...,워싱턴으로 진군하면서 해야 할 일이 무엇인지 정확히 알고 있었다.,contradiction


In [8]:
test_data.head()

,sentence1,sentence2,gold_label
0,"글쎄, 나는 그것에 관해 생각조차 하지 않았지만, 나는 너무 좌절했고, 결국 그에게...",나는 그와 다시 이야기하지 않았다.,contradiction
1,"글쎄, 나는 그것에 관해 생각조차 하지 않았지만, 나는 너무 좌절했고, 결국 그에게...",나는 다시 그와 이야기를 하기 시작했다는 것에 너무 화가 났다.,entailment
2,"글쎄, 나는 그것에 관해 생각조차 하지 않았지만, 나는 너무 좌절했고, 결국 그에게...",우리는 좋은 대화를 나눴다.,neutral
3,"그리고 저는 그것이 특권이라고 생각했습니다, 그리고 여전히, 여전히, 당시 저는 A...",그날 현장에 나만 있었던 게 아니라는 걸 몰랐던 것이다.,neutral
4,"그리고 저는 그것이 특권이라고 생각했습니다, 그리고 여전히, 여전히, 당시 저는 A...",나는 AFFC 공군 경력 분야에서 그 번호를 가진 유일한 사람이라는 인상을 가지고 ...,entailment


훈련 데이터, 검증 데이터, 테스트 데이터에 중복 샘플이나 결측값이 있는 샘플이 있다면 제거해줍니다.
이는 dropn_na_and_duplciates 라는 함수를 사용하여 적용합니다.

In [9]:
def drop_na_and_duplciates(df):
  df = df.dropna()
  df = df.drop_duplicates()
  df = df.reset_index(drop=True)
  return df

In [10]:
# 결 측 값 및 중 복 샘 플 제 거
train_data = drop_na_and_duplciates(train_data)
val_data = drop_na_and_duplciates(val_data)
test_data = drop_na_and_duplciates(test_data)

In [11]:
# 각 샘플 수
print('훈 련 용 샘 플 개 수 :',len(train_data))
print('검 증 용 샘 플 개 수 :',len(val_data))
print('테 스 트 용 샘 플 개 수 :',len(test_data))

훈 련 용 샘 플 개 수 : 941814
검 증 용 샘 플 개 수 : 2490
테 스 트 용 샘 플 개 수 : 5010


###2. GPT의 입력
토큰화를 위해 KoGPT‐2 토크나이저를 로드합니다.


In [12]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='<s>',
eos_token='</s>', pad_token='<pad>')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [13]:
# KoGPT‐2 의 정수 0, 1, 2, 3, 4 는 각각 어떤 토큰을 의미하는지 확인해봅시다.
print(tokenizer.decode(0))
print(tokenizer.decode(1))
print(tokenizer.decode(2))
print(tokenizer.decode(3))
print(tokenizer.decode(4))

<s>
</s>
<usr>
<pad>
<sys>


임의의 샘플을 작성하여 토크나이저의 전처리 결과를 봅시다. 여기서는 최대 길이를 128 로 정했습니
다.

In [14]:
max_seq_len = 128
sent1 = '모든 섬사람들이 알고 있듯이, 가장 멋진 만과 해변들 중 많은 것들은 보트로만 도달 할 수 있다'
sent2 = '보트는 일부 지역으로 가는 유일한 여행 수단이다.'

print('문장1: ', sent1)
print('문장2: ', sent2)

문장1:  모든 섬사람들이 알고 있듯이, 가장 멋진 만과 해변들 중 많은 것들은 보트로만 도달 할 수 있다
문장2:  보트는 일부 지역으로 가는 유일한 여행 수단이다.


위의 두 개의 문장으로부터 KoGPT‐2 에 넣을 입력으로 전처리를 진행해봅시다. KoGPT‐2 가 두 개의 서로
다른 문장임을 인식할 수 있도록 힌트를 줄 필요가 있습니다. 저자는 각 문장의 시작과 끝에 KoGPT‐2 의
시작 토큰과 종료 토큰을 붙이는 방식을 택했습니다. 그리고 입력이 완전히 끝났다는 것을 알려주기 위해
서 <unused0>라는 사용 용도가 정해져 있지 않은 KoGPT‐2 의 스페셜 토큰을 사용하였습니다. 그 후 배
치 연산을 위해 패딩을 해줍니다. KoGPT‐2 의 패딩 토큰은 정수 3 입니다.

In [15]:
bos_token = [tokenizer.bos_token]
eos_token = [tokenizer.eos_token]

# 첫 문장의 앞과 뒤에 시작 토큰 <s>와 종류 토큰 </s>으로 감싼다.
sent1_tokens = bos_token + tokenizer.tokenize(sent1) + eos_token

# 두번째 문장의 앞과 뒤에 시작 토큰 <s>와 종류 토큰 </s>으로 감싼뒤 <unused0>을 붙인다
sent2_tokens = bos_token + tokenizer.tokenize(sent2) + eos_token + ['<unused0>']

# 두 개의 문장을 연달아 이어 붙인 후 정수 인코딩을 수행한다.
tokens = sent1_tokens +sent2_tokens
input_id = tokenizer.convert_tokens_to_ids(tokens)
print('정수 인코딩 전: ', tokens)
print('정수 인코딩 후: ', input_id)

# 최대 길이로 패딩
input_id = pad_sequences([input_id], maxlen= max_seq_len, value=tokenizer.pad_token_id,
                        padding='post')[0]
print('패딩 후: ', input_id)

정수 인코딩 전:  ['<s>', '▁모든', '▁섬', '사람들이', '▁알고', '▁있듯이,', '▁가장', '▁멋진', '▁만과', '▁해변', '들', '▁중', '▁많은', '▁것들은', '▁보', '트로', '만', '▁도달', '▁할', '▁수', '▁있다', '</s>', '<s>', '▁보', '트는', '▁일부', '▁지역으로', '▁가는', '▁유일한', '▁여행', '▁수단이', '다.', '</s>', '<unused0>']
정수 인코딩 후:  [0, 9548, 9709, 34539, 12487, 42370, 9278, 43719, 34766, 23545, 7285, 9044, 9366, 24860, 9049, 11714, 7489, 14829, 9337, 9025, 33400, 1, 0, 9049, 11943, 9616, 14303, 11318, 13382, 12079, 26626, 9016, 1, 9]
패딩 후:  [    0  9548  9709 34539 12487 42370  9278 43719 34766 23545  7285  9044
  9366 24860  9049 11714  7489 14829  9337  9025 33400     1     0  9049
 11943  9616 14303 11318 13382 12079 26626  9016     1     9     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3

전처리 결과는 위와 같습니다. 각 문장의 시작에는 시작 토큰인 <s<e>>가 붙어 있으며 정수로는 1입니다. 이에 따라 두 개의 문장의 앞, 뒤에는 0과 1이 있습니다. 그리고 입력이 완전히 끝니면<unused0<e>>에 해당하는 정수인 9가 부착됩니다. 그리고 최대 길이를 128로 정하였으므로 128의 길이로 일치 시켜주기 위해서 패딩 토큰인 정수3이 채워집니다.

과정을 convert_examples_to_features 라는 함수로 만들고, 훈련 데이터의 첫번째 샘플을 가지고 임
의로 진행했던 전처리를 훈련 데이터, 검증 데이터, 테스트 데이터에 대해서 모두 진행해봅시다.

In [16]:
def convert_examples_to_features(sent_list1, sent_list2, max_seq_len, tokenizer):
  input_ids = []

  for sent1, sent2 in tqdm(zip(sent_list1, sent_list2), total=len(sent_list1)):
    bos_token = [tokenizer.bos_token]
    eos_token = [tokenizer.eos_token]
    sent1_tokens = bos_token + tokenizer.tokenize(sent1) + eos_token
    sent2_tokens = bos_token + tokenizer.tokenize(sent2) + eos_token + ['<unused0>']

    tokens = sent1_tokens + sent2_tokens
    input_id = tokenizer.convert_tokens_to_ids(tokens)
    input_id = pad_sequences([input_id], maxlen=max_seq_len, value=tokenizer.
    pad_token_id, padding='post')[0]

    assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
    input_ids.append(input_id)

  input_ids = np.array(input_ids, dtype=int)
  return input_ids

훈련 데이터에 대해서 전처리를 진행합니다.

In [17]:
X_train = convert_examples_to_features(train_data['sentence1'], train_data['sentence2'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 941814/941814 [03:37<00:00, 4338.35it/s]


훈련 데이터의 첫번째 샘플에 대한 정수 인코딩, 정수 인코딩을 기존의 문자열로 복원한 결과는 다음과 같
습니다. (데이터가 섞일 때는 랜덤이므로 저자와 첫번째 샘플은 다를 수 있습니다.)

In [18]:
input_id = X_train[0]

print('단어 정수 인코딩: ', input_id)
print('각 인코딩 길이: ', len(input_id))
print('정수 인코딩 디코딩: ', tokenizer.decode(input_id))

단어 정수 인코딩:  [    0  9174 25701 33755  8599 10371  9080 11539  7608  8137 13602 44766
  9016     1     0  9174 25701 23874 10721  7383  8165  9368  9676 10960
     1     9     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3]
각 인코딩 길이:  128
정수 인코딩 디코딩:  <s> 두 아이가 텐트 뒤에 있는 흙밭을 뛰어다닌다.</s><s> 두 아이가 밖에서 술래잡기를 하고 있다.</s><unused0><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

검증 데이터에 대해서도 전처리를 진행해봅시다.

In [19]:
X_val = convert_examples_to_features(val_data['sentence1'], val_data['sentence2'],max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 2490/2490 [00:00<00:00, 4038.08it/s]


검증 데이터의 첫번째 샘플에 대해서 전처리를 진행한 결과는 다음과 같습니다.

In [20]:
input_id = X_val[0]

print('단어 정수 인코딩: ', input_id)
print('각 인코딩 길이: ', len(input_id))
print('정수 인코딩 디코딩: ', tokenizer.decode(input_id))

단어 정수 인코딩:  [    0  9394  9871  9135  8718 14364 10063  8013 37144  9265 12583  8006
 25856   377     1     0  9258 10192  9848 11001 10644 10396 18796 20485
 37472  9134 35673  9539 18174     1     9     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     3]
각 인코딩 길이:  128
정수 인코딩 디코딩:  <s> 그리고 그가 말했다, "엄마, 저 왔어요."</s><s> 그는 학교 버스가 그를 내려주자마자 엄마에게 전화를 걸었다.</s><unused0><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

테스트 데이터에 대해서 전처리를 진행해봅시다.

In [21]:
X_test = convert_examples_to_features(test_data['sentence1'], test_data['sentence2'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 5010/5010 [00:01<00:00, 3992.80it/s]


contradiction, entailment, neutral 과 같이 문자열로 구성된 레이블에 대해서도 정수 인코딩을 진행합니
다.

In [22]:
train_label = train_data['gold_label'].tolist()
val_label = val_data['gold_label'].tolist()
test_label = test_data['gold_label'].tolist()

le = preprocessing.LabelEncoder()
le.fit(train_label)

# 고유한 정수로 변환
y_train = le.transform(train_label)
y_val = le.transform(val_label)
y_test = le.transform(test_label)

label_idx = dict(zip(list(le.classes_), le.transform(list(le.classes_))))
idx_label = {value: key for key, value in label_idx.items()}
print('각 레이블과 정수: ', label_idx)


각 레이블과 정수:  {np.str_('contradiction'): np.int64(0), np.str_('entailment'): np.int64(1), np.str_('neutral'): np.int64(2)}


레이블을 정수 인코딩을 하기 전과 후를 테스트 데이터에 대해서 상위 5 개를 출력하여 비교해봅시다.

In [23]:
print('변 환 전 :', test_label[:5])
print('변 환 후 :',y_test[:5])

변 환 전 : ['contradiction', 'entailment', 'neutral', 'neutral', 'entailment']
변 환 후 : [0 1 2 2 1]


###3. GPT의 출력 이해하기
koGPT‐2 를 이용해 모델을 구현하기 위해서는 koGPT‐2 의 출력을 이해할 필요가 있습니다. 우선, 한국어
GPT‐2 인 skt/kogpt2‐base‐v2 를 로드해봅시다.

In [24]:
model= TFGPT2Model.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.7.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'lm_head.weight']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from 

koGPT‐2 의 출력을 outpus 이라는 변수에 저장합니다. 입력 문장의 길이는 128 로 가정합니다.

In [25]:
max_seq_len = 128

# input_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
# outputs = model(input_ids_layer)

outputs 에는 두 개의 출력이 존재하는데 인덱스 0 을 확인해봅시다.

In [26]:
# # 문장 길이만큼의 출력
# print(outputs[0])

outputs[0] 은 (배치 크기, 128, 768) 의 크기를 가지는 텐서입니다. 이는 768 차원의 벡터가 128 개가 있다
는 의미로 문장 길이 개수만큼의 출력을 얻었음을 의미합니다. 텍스트 분류 문제를 풀 경우에는 koGPT‐2
의 마지막 예측에 해당하는 벡터를 사용해야 합니다.

In [27]:
# print(outputs[0][:,-1])

###4.GPT를 이용한 다중 클래스 분류 모델
서브클래싱 구현 방식으로 구현한 텍스트 분류 모델은 다음과 같습니다. GPT의 출력중 outputs[0][:,-1]. 즉, 마지막 출력 벡터를 소프트맥스 함수가 활성화 함수로 설정된 출력층으로 연결합니다.

In [28]:
class TFGPT2ForSequenceClassification(tf.keras.Model):
  def __init__(self, model_name, num_labels):
    super(TFGPT2ForSequenceClassification, self).__init__()
    self.gpt = TFGPT2Model.from_pretrained(model_name, from_pt=True)
    self.classifier = tf.keras.layers.Dense(num_labels,
                                            kernel_initializer=tf.keras.
                                            initializers.TruncatedNormal(0.02),
                                            activation='softmax', name='classifier')

  def call(self, inputs):
    outputs = self.gpt(input_ids=inputs)
    cls_token = outputs[0][:,-1]
    prediction = self.classifier(cls_token)

    return prediction

모델을 컴파일 합니다. 토크나이저와 동일하게 skt/kogpt2-base-v2를 사용하며 레이블의 수는 3입니다.

In [29]:
model = TFGPT2ForSequenceClassification("skt/kogpt2-base-v2", num_labels=3)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.7.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'lm_head.weight']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

두번의 epochs를 사용하여 정확도가 낮게 나오지만 epoch를 10~15 정도 설정하면 높게 나올거라 예상됩니다.

In [30]:
eary_stopping =EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.001,
    patience = 2
)

model.fit(
    X_train, y_train, epochs=2, batch_size=32, validation_data=(X_val, y_val),
    callbacks = [eary_stopping]
)

Epoch 1/2
29432/29432 ━━━━━━━━━━━━━━━━━━━━ 429s 14ms/step - accuracy: 0.3669 - loss: 1.1010 - val_accuracy: 0.3807 - val_loss: 1.0935
Epoch 2/2
29432/29432 ━━━━━━━━━━━━━━━━━━━━ 402s 14ms/step - accuracy: 0.4019 - loss: 1.0813 - val_accuracy: 0.3871 - val_loss: 1.0897


In [31]:
results = model.evaluate(X_test, y_test, batch_size=32)
print("test Loss, test Acc: ", results)

157/157 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 0.3925 - loss: 1.0889
test Loss, test Acc:  [1.088650107383728, 0.3944111764431]
